In [ ]:
!pip install pinecone
!pip install openai

In [ ]:
from pinecone import Pinecone, ServerlessSpec
import os
from openai import OpenAI
import pandas as pd
from time import time
import dotenv
dotenv.load_dotenv()

In [ ]:
token= os.getenv("OPENAI_API_KEY") 
open_ai_base_url = os.getenv("RUNPOD_CHATBOT_URL") 
model_name= os.getenv("MODEL_NAME") 
pinecone_api_key = os.getenv("PINECONE_API_KEY")

In [ ]:
pc = Pinecone(api_key=pinecone_api_key)

client = OpenAI(
  api_key=token, 
)

In [ ]:
output = client.embeddings.create(input = ["helloo there"],model="text-embedding-ada-002")
embedings = output.data[0].embedding
print(embedings)

In [ ]:
len(embedings)

In [22]:
df=pd.read_json('D:/project/Vietnamese-Restaurant-Service-Chatbot/products/vietnamese_food_detail.jsonl',lines=True)

In [23]:
df.head(2)

,name,category,description,ingredients,price,rating,image_path
0,Pho Ga,Noodles,A delicious Vietnamese noodle soup with tender...,"[Chicken, Rice Noodles, Broth, Herbs]",6.5,4.8,pho_ga.jpg
1,Goi Cuon,Seafood,Fresh Vietnamese spring rolls filled with shri...,"[Shrimp, Rice Paper, Vermicelli, Herbs]",5.0,4.7,goi_cuon.jpg


In [24]:
df['text'] =  df['name']+" : "+df['description'] + \
                " -- Ingredients: " + df['ingredients'].astype(str) + \
                " -- Price: " + df['price'].astype(str) + \
                " -- rating: " + df['rating'].astype(str) 

In [25]:
df['text'].head()

0    Pho Ga : A delicious Vietnamese noodle soup wi...
1    Goi Cuon : Fresh Vietnamese spring rolls fille...
2    Hu Tieu Nam Vang : A flavorful Vietnamese nood...
3    Tra Da : A refreshing Vietnamese iced tea made...
4    Nuoc Mia : A sweet and refreshing Vietnamese s...
Name: text, dtype: object

In [29]:
texts = df['text'].tolist()
with open('D:/project/Vietnamese-Restaurant-Service-Chatbot/products/info.txt') as f:
    Mrs_Phuong_Cozzy = f.read()
    
Mrs_Phuong_Cozzy = "Mrs.Phuong Cozzy" + Mrs_Phuong_Cozzy
texts.append(Mrs_Phuong_Cozzy)

In [31]:
with open('D:/project/Vietnamese-Restaurant-Service-Chatbot/products/menu.txt') as f:
    menue_items_text = f.read()
    
menue_items_text = "Menu Items: " + menue_items_text
texts.append(menue_items_text)

In [33]:
output = client.embeddings.create(input = texts,model="text-embedding-ada-002")
embeddings = output.data

In [34]:
index_name

NameError: name 'index_name' is not defined

In [42]:
index_name = "vietnamesecozzyrestaurant"

pc.create_index(
    name=index_name,
    dimension=1536, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

{
    "name": "vietnamesecozzyrestaurant",
    "metric": "cosine",
    "host": "vietnamesecozzyrestaurant-j6lotk8.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": null
}

In [43]:
import hashlib
import time
import pinecone

# Initialize Pinecone client
pc = pinecone.Pinecone(api_key=pinecone_api_key)  # Ensure env var or use directly

# Wait for index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)

# Prepare vectors
vectors = []
for text, embedding in zip(texts, embeddings):
    # Create a safe, unique ID using MD5 hash
    entry_id = hashlib.md5(text.encode('utf-8')).hexdigest()
    
    # Optional: validate shape of embedding if needed
    if hasattr(embedding, "embedding"):
        values = embedding.embedding
    else:
        values = embedding  # In case you're already passing raw vector

    vectors.append({
        "id": entry_id,
        "values": values,
        "metadata": {'text': text}
    })

# Upsert to Pinecone
response = index.upsert(
    vectors=vectors,
    namespace="ns1"
)

print("Upsert response:", response)


Upsert response: {'upserted_count': 33}


In [44]:
output = client.embeddings.create(input = ["Is Pho Ga lactose-free?"],model="text-embedding-ada-002")
embeding = output.data[0].embedding

In [45]:
results = index.query(
    namespace="ns1",
    vector=embeding,
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)

{'matches': [{'id': '974fa0f5d9249d1fbede757866a880f1',
              'metadata': {'text': 'Pho Ga : A delicious Vietnamese noodle '
                                   'soup with tender chicken, fresh herbs, and '
                                   'rice noodles in a savory broth. -- '
                                   "Ingredients: ['Chicken', 'Rice Noodles', "
                                   "'Broth', 'Herbs'] -- Price: 6.5 -- rating: "
                                   '4.8'},
              'score': 0.843781,
              'values': []},
             {'id': '3700e16a08552de884ec144bff4a496e',
              'metadata': {'text': 'Menu Items: Menu\n'
                                   '\n'
                                   'Pho Ga - $6.50\n'
                                   'Goi Cuon - $5.00\n'
                                   'Hu Tieu Nam Vang - $7.00\n'
                                   'Tra Da - $1.50\n'
                                   'Nuoc Mia - $2.00\n'
            